In [1]:
Field = GF(2)
PolyRing.<S> = PolynomialRing(Field)
var('D', 'I')

# [D^2, D^2+D, D^2+D+1]
gens = [
    S^2,
    S^2 + S,
    S^2 + S + 1
]

degrees = list(map(lambda x: x.degree(), gens))

In [2]:
import itertools as it
import numpy as np

reg_len = max(degrees)

def reg_to_str(reg): return "".join(map(str, reg))

edges = {}
all_ger_states = list(it.product([0, 1], repeat=reg_len))
for reg_state in all_ger_states:
    vec = reg_to_str(reg_state)
    edges[vec] = {}
    
def calc_outputs(adjasted_reg):
    res_vec = []
    for gen in gens:
        gg = gen.list()
        gg = gg + [0] * (reg_len - len(gg) + 1)
        res_vec.append(np.dot(adjasted_reg, gg))
    return res_vec
    
def process_node(reg_state):
    vec = reg_to_str(reg_state)
    if len(edges[vec]) > 0: 
        return
    
    reg0 = [0] + reg_state
    next_node = reg0[:-1]
    outs = calc_outputs(reg0)
    edges[vec][0] = (reg_to_str(next_node), outs, D ^ sum(map(int, outs)))
    process_node(next_node)
    
    reg1 = [1] + reg_state
    next_node = reg1[:-1]
    outs = calc_outputs(reg1)
    edges[vec][1] = (reg_to_str(next_node), outs, D ^ sum(map(int, outs)))
#     edges[vec][1] = (reg_to_str(next_node), outs, I * D ^ sum(map(int, outs)))
    process_node(next_node)
    
process_node([0] * reg_len)
print(edges)

{'00': {0: ('00', [0, 0, 0], 1), 1: ('10', [0, 0, 1], D)}, '01': {0: ('00', [1, 1, 1], D^3), 1: ('10', [1, 1, 0], D^2)}, '10': {0: ('01', [0, 1, 1], D^2), 1: ('11', [0, 1, 0], D)}, '11': {0: ('01', [1, 0, 0], D), 1: ('11', [1, 0, 1], D^2)}}


In [3]:
import graphviz as gv

def printGraph(edges):
    gr = gv.Digraph(engine='dot')
    for reg_state in all_ger_states:
        as_str = reg_to_str(reg_state)
        gr.node(as_str)

    for reg_state in all_ger_states:
        as_str = reg_to_str(reg_state)
        (next_node, elems, err) = edges[as_str][0]
        gr.edge(as_str, next_node, label=f'0/{reg_to_str(elems)}/{err}')
        
        (next_node, elems, err) = edges[as_str][1]
        gr.edge(as_str, next_node, label=f'1/{reg_to_str(elems)}/{err}', color='orange', fontcolor='orange')
    return gr

gr = printGraph(edges)

In [4]:
gr.view()

'Digraph.gv.pdf'

In [5]:
all_ger_len = len(all_ger_states)
A = [[0 for j in range(all_ger_len)] for i in range(all_ger_len)]
b = [0 for i in range(all_ger_len)]

for reg_state in all_ger_states:
    as_str = reg_to_str(reg_state)
    num = int(as_str, 2)
    A[num][num] = 1
    
    (to0, _, err) = edges[as_str][0]
    if as_str != to0:
        num_to0 = int(to0, 2)
        if num == 0:
            b[num_to0] += err
        else:
            A[num_to0][num] -= err
    
    (to1, _, err) = edges[as_str][1]
    num_to1 = int(to1, 2)
    if num == 0:
        b[num_to1] += err
    else:
        A[num_to1][num] -= err

print(np.array(A))
print(b)

[[1 -D^3 0 0]
 [0 1 -D^2 -D]
 [0 -D^2 1 0]
 [0 0 -D -D^2 + 1]]
[0, 0, D, 0]


In [6]:
solution = Matrix(A).solve_right(vector(b))
for e in solution:
    print(e.full_simplify())

-(D^8 - 2*D^6)/(D^6 - 2*D^4 - D^2 + 1)
-(D^5 - 2*D^3)/(D^6 - 2*D^4 - D^2 + 1)
-(D^3 - D)/(D^6 - 2*D^4 - D^2 + 1)
D^2/(D^6 - 2*D^4 - D^2 + 1)


In [7]:
solution[0].series(D, 15)
# 2 0 1 0 5

2*D^6 + 1*D^8 + 5*D^10 + 5*D^12 + 14*D^14 + Order(D^15)

In [8]:
ndi = 3 * D^6 * I^2 + D^5 * I
mdi = 3 * D^5 * I + 4 * D^3 * I


tdi = ndi / (1-mdi)
tdi.subs(I = 1).series(D, 10)
# 1 3 0 4 12

1*D^5 + 3*D^6 + 4*D^8 + 12*D^9 + Order(D^10)

In [9]:
tdi.derivative(I).subs(I = 1).series(D, 10)
# 1 6 0 8 36

1*D^5 + 6*D^6 + 8*D^8 + 36*D^9 + Order(D^10)